In [3]:
from components.DataCollection import DataCollection
import pandas as pd
import numpy as np
import datetime
from statsmodels.tsa.stattools import coint, adfuller
import matplotlib.pyplot as plt
data = DataCollection()

Get all NYSE tickers, with closing price under $75

In [2]:

tickers = data.all_nyse_tickers()
tickers = [ticker for ticker in tickers if '$' not in ticker and '.' not in ticker]
filtered_tickers = data.filter_last_closing_price(75, tickers, 5)

print(len(tickers))

CBO: No data found for this date range, symbol may be delisted
CBX: No data found, symbol may be delisted
NTEST: No data found for this date range, symbol may be delisted
2439


In [3]:
#get 30 day stationarity of log returns
lookback = 30
stationary_tickers = []

for ticker in filtered_tickers:
    print('getting stationarity of ticker ' + ticker)
    prices = data.closing_prices(ticker, datetime.date.today() - datetime.timedelta(days=lookback),datetime.date.today())
    returns = data.log_returns(prices)

    try:
        reverting = adfuller(returns)
        pvalue = reverting[1]
        critical_value = reverting[4]['1%']
        test_statistic = reverting[0]

        if pvalue < 0.05 and test_statistic < critical_value:
            print('stationary')
            stationary_tickers.append(ticker)
    except:
        print('didnt work lol')



getting stationarity of ticker AA
getting stationarity of ticker AACT
stationary
getting stationarity of ticker AAN
getting stationarity of ticker AAP
getting stationarity of ticker AAT
getting stationarity of ticker AB
getting stationarity of ticker ABCB
getting stationarity of ticker ABEV
getting stationarity of ticker ABM
getting stationarity of ticker ABR
getting stationarity of ticker AC
getting stationarity of ticker ACCO
getting stationarity of ticker ACEL
getting stationarity of ticker ACHR
getting stationarity of ticker ACI
getting stationarity of ticker ACP
getting stationarity of ticker ACR
stationary
getting stationarity of ticker ACRE
stationary
getting stationarity of ticker ACV
getting stationarity of ticker ADCT
getting stationarity of ticker ADM
getting stationarity of ticker ADNT
getting stationarity of ticker ADT
stationary
getting stationarity of ticker ADX
getting stationarity of ticker AEFC
getting stationarity of ticker AEG
getting stationarity of ticker AEO
gett

now we have a list of tickers filtered by price and stationarity of log returns between 30 days ago and now

In [4]:
print(len(stationary_tickers))

628


filter by volume >= 200k and volatility >= 10%

In [17]:
filtered_tickers = []
for i, ticker in enumerate(stationary_tickers):
    price_data = data.ohlcv(ticker, datetime.date.today() - datetime.timedelta(days=lookback), datetime.date.today())
    avg_volume = np.mean(price_data['Volume'])
    volatility = np.std(price_data['Close'])
    if avg_volume > 200000 and volatility > 0.1: filtered_tickers.append(ticker)


1 Failed download:
- DXYZ: No data found for this date range, symbol may be delisted


In [18]:
print(len(filtered_tickers))
data.write_to_txt_file(filtered_tickers, 'low_cap_stationary_tickers_' + str(datetime.date.today()))

320
wrote tickers to file low_cap_stationary_tickers_2024-09-14


In [5]:
print(data.get_risk_free_rate())

0.03620999813079834
